If you're named Danny Kyung or Matthew Emes, it opens up the possibility of justifying your use of usernames such as [dank](https://github.com/dank) or [memes](https://github.com/memes).

Your task is to find the longest word such that it satisfies the criteria - that is, it is a substring of the given string but not necessarily consecutively (we can call it a sparse substring). If there are multiple words of same maximum length, output all of them.

You may use the the [Enable word list](http://norvig.com/ngrams/enable1.txt), or some other reasonable English word list. Every word in your output must appear in your word list.
Formal Inputs & Outputs

#### Example Inputs
```
Donald Knuth
Alan Turing
Claude Shannon
```

#### Output description
```
A single word, or the lengthiest word/words in case of multiple words satisfying the criteria.
```

#### Example outputs
```
Donut (because **Don**ald k**nut**h)
Alanin, Anting
Cannon
```

#### Challenge Inputs
```
Ada Lovelace
Haskell Curry
**Your own name!**
```

In [1]:
from itertools import chain, combinations
import re

import pandas as pd

url = 'http://norvig.com/ngrams/enable1.txt'
words = set(pd.read_csv(url, header=None, squeeze=True).dropna())


def powerset(iterable):
    """From itertools recipes."""
    # https://docs.python.org/3/library/itertools.html#itertools-recipes
    # powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    # This is, admittedly, far from the fastest way about things
    s = tuple(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))


def check_substrings(phrase):
    phrase = re.sub('[^a-z]', '', phrase.lower())
    substrings = powerset(phrase)
    matches = set(''.join(el) for el in substrings).intersection(words)
    maxlength = max(map(len, matches))
    res = tuple(match for match in matches if len(match) == maxlength)
    return res[0] if len(res) == 1 else res

inputs = [
    'Donald Knuth',
    'Alan Turing',
    'Claude Shannon',
    'Ada Lovelace',
    'Haskell Curry',
    'Bradley Joseph Solomon',
    'Audrey Elizabeth Baker',
    'Kevin Stern'
    ]

In [2]:
def with_itertools():
    for i in inputs:
        check_substrings(i)
        
%timeit with_itertools()

1.06 s ± 17.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
def substring_positions(phrase, word):
    # Assumes phrase is lower() + ascii lowercase only
    pos = 0
    for w in word:
        p = phrase[pos:].index(w) + pos
        yield p
        pos = p + 1

def is_monotonic_incr(obj):
    if not obj:
        # Handle []
        return False
    return all(i < j for i, j in zip(obj, obj[1:]))

def is_substring(phrase, word):
    try:
        pos = list(substring_positions(phrase, word))
    except:
        return False
    return is_monotonic_incr(pos)

def check_substrings(phrase):
    phrase = re.sub('[^a-z]', '', phrase.lower())
    matches = []
    for w in words:
        if is_substring(phrase, w):
            matches.append(w)
    maxlength = max(map(len, matches))
    res = tuple(match for match in matches if len(match) == maxlength)
    return res[0] if len(res) == 1 else res

In [4]:
for i in inputs:
    print(check_substrings(i))

donut
('alanin', 'anting', 'luring')
('cannon', 'clades')
dolce
('slurry', 'scurry', 'skerry')
('bradoon', 'blossom', 'braless', 'bassoon', 'balloon', 'josephs')
('retaker', 'diether', 'relater', 'drabber', 'aureate', 'eyelike')
keister


In [5]:
def no_itertools():
    for i in inputs:
        check_substrings(i)
        
%timeit no_itertools()

1.79 s ± 47.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit check_substrings('Alan Turing Jean Claude Vandame')

272 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
